
# グローバル ベクトルを使用した単語埋め込み (GloVe)

 :ラベル: `sec_glove`

コンテキスト ウィンドウ内での単語間の共起には、豊富な意味情報が含まれる場合があります。たとえば、大規模なコーパスでは、単語「固体」は「蒸気」よりも「氷」と共起する可能性が高くなりますが、単語「ガス」はおそらく「氷」よりも「蒸気」と共起する可能性が高くなります。さらに、そのような共起のグローバルなコーパス統計を事前に計算することができるため、より効率的なトレーニングにつながる可能性があります。単語の埋め込みにコーパス全体の統計情報を活用するには、まず :numref: `subsec_skip-gram`のスキップグラム モデルを再検討しますが、共起数などのグローバル コーパス統計を使用して解釈します。

## グローバル コーパス統計によるスキップグラム

:label: `subsec_skipgram-global`

スキップグラムモデルの単語 $w_i$ が与えられた単語 $w_j$ の条件付き確率 $P(w_j\mid w_i)$ を $q_{ij}$ で表すと、次のようになります。

 $$q_{ij}=\frac{\exp(\mathbf{u} *j^\top \mathbf{v}_i)}{ \sum* {k \in \mathcal{V}} \text{exp}(\ mathbf{u}_k^\top \mathbf{v}_i)},$$

ここで、任意のインデックス $i$ ベクトル $\mathbf{v}_i$ と $\mathbf{u}_i$ は、単語 $w_i$ をそれぞれ中心単語と文脈単語として表し、$\mathcal{V} = {0 , 1, \ldots, |\mathcal{V}|-1}$ は語彙のインデックス セットです。

コーパス内に複数回出現する可能性がある単語 $w_i$ を考えてみましょう。コーパス全体において、$w_i$ が中心単語として取られるすべての文脈単語は*、同じ要素の複数のインスタンスを許可する*単語インデックスの*マルチセット*$\mathcal{C}_i$ を形成します。どの要素でも、そのインスタンスの数は*多重度*と呼ばれます。例で説明すると、単語 $w_i$ がコーパス内に 2 回出現し、2 つのコンテキスト ウィンドウで $w_i$ を中心単語とするコンテキスト単語のインデックスが $k、j、m、k$、$k であるとします。 、l、k、j$。したがって、マルチセット $\mathcal{C}_i = {j, j, k, k, k, k, l, m}$、ここで要素 $j, k, l, m$ の多重度は 2, 4, 1,それぞれ1。

ここで、マルチセット $\mathcal{C} *i$ の要素 $j$ の多重度を $x* {ij}$ と表しましょう。これは、コーパス全体の同じ文脈ウィンドウ内の単語 $w_j$ (文脈単語として) と単語 $w_i$ (中心単語として) のグローバル共起数です。このようなグローバル コーパス統計を使用すると、スキップグラム モデルの損失関数は次のようになります。

 $$-\sum_{i\in\mathcal{V}}\sum_{j\in\mathcal{V}} x_{ij} \log,q_{ij}.$$ :eqlabel: `eq_skipgram-x_ij`

さらに、$w_i$ が中心単語として出現する文脈ウィンドウ内のすべての文脈単語の数を $x_i$ で示します。これは、$|\mathcal{C} *i|$ と同等です。 $p* {ij}$ を中心単語 $w_i$ から文脈単語 $w_j$ を生成するための条件付き確率 $x_{ij}/x_i$ とすると、:eqref: `eq_skipgram-x_ij`次のように書き換えることができます。

 $$-\sum_{i\in\mathcal{V}} x_i \sum_{j\in\mathcal{V}} p_{ij} \log,q_{ij}.$$ :eqlabel: `eq_skipgram-p_ij`

 :eqref: `eq_skipgram-p_ij`では、 $-\sum_{j\in\mathcal{V}} p_{ij} \log,q_{ij}$ は条件付き分布 $p_{ij}$ のクロスエントロピーを計算します。グローバルコーパス統計とモデル予測の条件付き分布$q_{ij}$。この損失も、上で説明したように $x_i$ によって重み付けされます。 :eqref: `eq_skipgram-p_ij`の損失関数を最小化すると、予測された条件付き分布をグローバル コーパス統計からの条件付き分布に近づけることができます。

クロスエントロピー損失関数は確率分布間の距離を測定するためによく使用されますが、ここでは適切な選択ではない可能性があります。一方で、 :numref: `sec_approx_train`で述べたように、 $q_{ij}$ を適切に正規化するコストは語彙全体の合計となり、計算コストが高くなる可能性があります。一方で、大規模なコーパスからの多数のまれなイベントは、多くの場合、クロスエントロピー損失によってモデル化され、過大な重みが割り当てられます。

## グローブモデル

これを考慮して、 *GloVe*モデルは二乗損失に基づいてスキップグラム モデルに 3 つの変更を加えています:cite: `Pennington.Socher.Manning.2014` :
1. 確率分布ではない変数 $p&#39; *{ij}=x* {ij}$ と $q *&#39; {ij}=\exp(\mathbf{u}_j^\top \mathbf{v}_i)$ を使用して、両方の対数なので、二乗損失項は $\left(\log,p&#39; {* ij} - \log,q *&#39; {ij}\right)^2 = \left(\mathbf{u}_j^\top \ mathbf{v}_i - \log,x* {ij}\right)^2$。
1. 各単語 $w_i$ に 2 つのスカラー モデル パラメーター (中心単語バイアス $b_i$ と文脈単語バイアス $c_i$) を追加します。
1. 各損失項の重みを重み関数 $h(x_{ij})$ に置き換えます。ここで、$h(x)$ は $[0, 1]$ の区間で増加します。

すべてをまとめると、GloVe のトレーニングは次の損失関数を最小限に抑えることになります。

 $$\sum_{i\in\mathcal{V}} \sum_{j\in\mathcal{V}} h(x_{ij}) \left(\mathbf{u} *j^\top \mathbf{v} _i + b_i + c_j - \log,x* {ij}\right)^2.$$ :eqlabel: `eq_glove-loss`

重み関数の場合、推奨される選択肢は次のとおりです。 $h(x) = (x/c) ^\alpha$ (例: $\alpha = 0.75$) if $x &lt; c$ (例: $c = 100$);それ以外の場合は $h(x) = 1$。この場合、$h(0)=0$ であるため、計算効率を高めるために、任意の $x_{ij}=0$ の二乗損失項を省略できます。たとえば、トレーニングにミニバッチの確率的勾配降下法を使用する場合、各反復で*ゼロ以外の*$x_{ij}$ のミニバッチをランダムにサンプリングして勾配を計算し、モデル パラメーターを更新します。これらのゼロ以外の $x_{ij}$ は、事前に計算されたグローバル コーパス統計であることに注意してください。したがって、このモデルは GloVe for *Global Vectors*と呼ばれます。

単語 $w_i$ が単語 $w_j$ のコンテキスト ウィンドウに表示される場合は、その*逆も同様であること*を強調しておく必要があります。したがって、$x_{ij}=x_{ji}$ となります。非対称の条件付き確率 $p_{ij}$ に適合する word2vec とは異なり、GloVe は対称 $\log , x_{ij}$ に適合します。したがって、GloVe モデルでは、任意の単語の中心単語ベクトルと文脈単語ベクトルは数学的に等価です。ただし、実際には、初期化値が異なるため、トレーニング後に同じ単語がこれら 2 つのベクトルで異なる値を取得する可能性があります。GloVe はそれらを出力ベクトルとして合計します。

## 共起確率の比からGloVeを解釈する

GloVe モデルを別の観点から解釈することもできます。 :numref: `subsec_skipgram-global`で同じ表記を使用して、 $p_{ij} \stackrel{\mathrm{def}}{=} P(w_j \mid w_i)$ をコンテキスト ワード $w_j$ を生成する条件付き確率とします。 $w_i$ をコーパスの中心単語として指定します。 :numref: `tab_glove`大規模なコーパスからの統計に基づいて、単語「ice」と「steam」に関するいくつかの共起確率とその比率をリストします。

 :大規模なコーパスからの単語間の共起確率とその比率 (:citet: `Pennington.Socher.Manning.2014`の表 1 から適応) :label: `tab_glove`

 |$w_k$=|固体|ガス|水|ファッション| |:--|:-|:-|:-|:-| |$p_1=P(w_k\mid \text{ice})$|0.00019|0.000066|0.003|0.000017| |$p_2=P(w_k\mid\text{steam})$|0.000022|0.00078|0.0022|0.000018| |$p_1/p_2$|8.9|0.085|1.36|0.96|

 :numref: `tab_glove`から次のことがわかります。
-  $w_k=\text{solid}$ など、「ice」に関連するが「steam」とは関係のない単語 $w_k$ の場合、8.9 など、より大きな共起確率比が期待されます。
-  $w_k=\text{gas}$ など、「蒸気」に関連するが「氷」とは関係のない単語 $w_k$ の場合、共起確率の比率は 0.085 など、より小さいと予想されます。
-  $w_k=\text{water}$ など、「氷」と「蒸気」の両方に関連する単語 $w_k$ の場合、共起確率の比は 1.36 など、1 に近いと予想されます。
-  $w_k=\text{fashion}$ など、「ice」と「steam」の両方に無関係な単語 $w_k$ の場合、共起確率の比が 1 に近い (0.96 など) と予想されます。

共起確率の比は単語間の関係を直感的に表現できることがわかります。したがって、この比率に適合するように 3 つのワード ベクトルの関数を設計できます。 $w_i$ を中心単語、$w_j$ と $w_k$ を文脈単語とした場合の共起確率 ${p_{ij}}/{p_{ik}}$ の比率をこの比率に適合させたいと考えます。関数 $f$ を使用して:

 $$f(\mathbf{ *u} j, \mathbf{u}_k, {\mathbf{v}}_i) \about \frac{p {* ij}}{p_{ik}}.$$ :eqlabel: `eq_glove-f`

 $f$ の考えられる多くの設計の中から、以下では合理的な選択のみを選択します。共起確率の比はスカラーであるため、$f$ は $f(\mathbf{u}_j, \mathbf{u}_k, {\mathbf{v}}_i など) のスカラー関数である必要があります。 ) = f\left((\mathbf{u}_j - \mathbf{u}_k)^\top {\mathbf{v}}_i\right)$. :eqref: `eq_glove-f`で単語インデックス $j$ と $k$ を切り替えると、 $f(x)f(-x)=1$ を保持する必要があるため、1 つの可能性は $f(x)=\exp(x )$、つまり、

 *$$f(\mathbf{u} j, \mathbf{u}_k, {\mathbf{v}}_i) = \frac{\exp\left(\mathbf{u}_j^\top {\mathbf{v }}_i\right)}{\exp\left(\mathbf{u}_k^\top {\mathbf{v}}_i\right)} \about \frac{p {ij}}{p_{ik}}* .$$

ここで $\exp\left(\mathbf{u *} j^\top {\mathbf{v}}_i\right) \about \alpha p* {ij}$ を選択しましょう。$\alpha$ は定数です。 $p_{ij}=x_{ij}/x_i$ なので、両辺の対数を取ると $\mathbf{u *} j^\top {\mathbf{v}}_i \about \log,\alpha + が*得られます。 *\log,x* {ij} - \log,x_i$。 $- \log, \alpha + \log, x_i$ を当てはめるために、中心単語バイアス $b_i$ や文脈単語バイアス $c_j$ など、追加のバイアス項を使用する場合があります。

 $$\mathbf{u} *j^\top \mathbf{v}_i + b_i + c_j \about \log, x* {ij}.$$ :eqlabel: `eq_glove-square`

 :eqref: `eq_glove-square`の二乗誤差を重み付きで測定すると、:eqref: `eq_glove-loss`の GloVe 損失関数が得られます。

## まとめ
- スキップグラム モデルは、単語間の共起数などのグローバル コーパス統計を使用して解釈できます。
- クロスエントロピー損失は、特に大規模なコーパスの場合、2 つの確率分布の差を測定するのには適切な選択ではない可能性があります。 GloVe は、二乗損失を使用して、事前に計算されたグローバル コーパス統計を適合させます。
- 中心単語ベクトルと文脈単語ベクトルは、GloVe のどの単語に対しても数学的に同等です。
-  GloVe は単語間の共起確率の比から解釈できます。

## 演習
1. 単語 $w_i$ と $w_j$ が同じコンテキスト ウィンドウ内で共起する場合、テキスト シーケンス内の単語の距離を使用して、条件付き確率 $p_{ij}$ を計算する方法を再設計するにはどうすればよいでしょうか?ヒント: GloVe 論文のセクション 4.2 を参照してください:cite: `Pennington.Socher.Manning.2014` 。
1. どのような単語でも、その中心単語のバイアスと文脈単語のバイアスは GloVe において数学的に同等ですか?なぜ？

[ディスカッション](https://discuss.d2l.ai/t/385)
